# Kép betöltése és megjelenítése

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
from scipy.signal import convolve2d #2D konvolúció
import os
from PIL import Image

In [ ]:
img = mpimg.imread('Images/cat.jpg')
imgplot = plt.imshow(img)
plt.show()


# Kép vágása

In [ ]:
#image shape (height, width, depth/color)
print(img.shape)

In [ ]:
img_cropped = img[0:600, 0:1000, :]
imgplot = plt.imshow(img_cropped)
plt.show()

In [ ]:
print(img_cropped.shape)

# Kép szürkeárnyalatossá tétele
## Átlagolással

In [ ]:
grey_img = cv2.imread('Images/cat.jpg')

#Transform so Matplotlib can read it correctly
fix_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.title('BGR konvertáció')
plt.imshow(fix_img)

#Matplotlib expects Red Green Blue
#OPENCV reads as Blue Green Red

#the three channels
R, G, B = fix_img[:,:,0], fix_img[:,:,1],fix_img[:,:,2]

In [ ]:
gray_avg_img = np.mean(fix_img, axis=2)
plt.imshow(gray_avg_img, cmap='gray')

## Súlyozott átlaggal 

In [ ]:
#OpenCV grayscale conversion formula
Y = 0.299 * R + 0.587 * G + 0.114 * B
print(Y)

In [ ]:
plt.imshow(Y, cmap='gray')

In [ ]:
#OpenCV function
gray_img_ocv = cv2.imread('Images/cat.jpg', cv2.IMREAD_GRAYSCALE)
#print(gray_img_ocv)

plt.imshow(gray_img_ocv, cmap='gray')

# Hisztogram
## OpenCV

In [ ]:
histg = cv2.calcHist([gray_img_ocv], [0], None, [256], [0, 256])
#cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])

plt.xlabel('Intenzitás')
plt.ylabel('Gyakoriság')
plt.title('Szürkeárnyalati hisztogram')
plt.hist(gray_img_ocv.ravel(), 256, [0,256])
plt.show()

## Matplotlib

In [ ]:
hist,bin = np.histogram(gray_avg_img.ravel(), 256, [0,255])

plt.plot(hist)
plt.xlabel('Intenzitás')
plt.ylabel('Gyakoriság')
plt.title('Szürkeárnyalati hisztogram')
plt.show()


# Küszöbölés

In [ ]:
_, thresholded_image = cv2.threshold(gray_avg_img, 127, 255, cv2.THRESH_BINARY)

plt.figure()
plt.imshow(thresholded_image, cmap='gray')
plt.axis('off')
plt.title('Globális küszöbölés')
plt.show()


# Átlagoló szűrő 
## OpenCV

In [ ]:
kernel_size = (3, 3)  # A szűrő mérete (pl. 3x3)
filtered_image = cv2.blur(img, kernel_size)

plt.figure()
plt.imshow(cv2.cvtColor(filtered_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Átlagoló szűrő (OpenCV)')
plt.show()

In [ ]:
def global_threshold_ratio(image, ratio):
    # Kép konvertálása szürkeárnyalatossá
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Küszöbérték meghatározása az adott arány alapján
    threshold_value = np.percentile(gray_image, ratio * 100)
    
    # Küszöbölés
    _, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    
    return thresholded_image


thresh_img = global_threshold_ratio(img, .6)


plt.figure()
plt.imshow(thresh_img, cmap='gray')
plt.axis('off')
plt.title('Globális küszöbölés arányosan')
plt.show()


In [ ]:
 '''Mivel a küszöbérték meghatározása a percentilis pont alapján történik,
 a pontos közelítés függ a kép intenzitáseloszlásától és a kiválasztott aránytól. 
 Általánosságban elmondható, hogy minél jobban közelít az intenzitáseloszlás az egyenletes eloszláshoz,
 annál pontosabb lesz a közelítés.'''

## Kézi implementáció

In [ ]:
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Kép átalakítása RGB formátumba

kernel_size = (3, 3)  # A szűrő mérete (pl. 3x3)
kernel = np.ones(kernel_size) / np.prod(kernel_size)

filtered_image_r = convolve2d(image_rgb[:,:,0], kernel, mode='same', boundary='symm')
filtered_image_g = convolve2d(image_rgb[:,:,1], kernel, mode='same', boundary='symm')
filtered_image_b = convolve2d(image_rgb[:,:,2], kernel, mode='same', boundary='symm')

filtered_image = np.stack((filtered_image_r, filtered_image_g, filtered_image_b), axis=2).astype(np.uint8)

plt.figure()
plt.imshow(filtered_image)
plt.axis('off')
plt.title('Átlagoló szűrő (kézi implementáció)')
plt.show()

## Különböző kernel méretekkel

In [ ]:
def average_filter(image, kernel_size):
    # Átlagoló szűrő alkalmazása
    filtered_image = cv2.blur(image, (kernel_size, kernel_size))
    return filtered_image

# Kép betöltése
img = cv2.imread('Images/blurry.jpg', cv2.IMREAD_GRAYSCALE)

# Kernel méretek léptetése
start = 3
stop = 101
step = 2
kernel_sizes = list(range(start, stop+1, step))

# Kimeneti mappa létrehozása
output_dir = 'average_filter'
os.makedirs(output_dir, exist_ok=True)

# Átlagolt képek elmentése különböző kernel méretekkel
for kernel_size in kernel_sizes:
    filtered_image = average_filter(img, kernel_size)
    file_name = f'{kernel_size}x{kernel_size}.png'
    file_path = os.path.join(output_dir, file_name)
    cv2.imwrite(file_path, filtered_image)


## Nem négyzetes kernel

In [ ]:

def average_filter(image, kernel):
    # Átlagoló szűrő alkalmazása
    filtered_image = cv2.filter2D(image, -1, kernel)
    
    return filtered_image

# Kernel definiálása
kernel = np.array([[1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9]])

'''A kernel értékeit lineárisan beállítottam 1/9-re, 
hogy megfeleljenek egy pontosabb átlagoló szűrőnek. 
Ez a kernel súlyozott átlagolást eredményez, amely nem módosítja a kép világosságát.'''


# Átlagoló szűrő alkalmazása nem négyzetes kernellel
filtered_image = average_filter(img, kernel)

# Eredmény megjelenítése
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axs[0].set_title('Eredeti kép')
axs[0].axis('off')

axs[1].imshow(cv2.cvtColor(filtered_image, cv2.COLOR_BGR2RGB))
axs[1].set_title('Átlagolt kép')
axs[1].axis('off')

plt.show()

# Medián szűrés

In [ ]:
def median_filter(image, kernel_size):
    # Mediánszűrő alkalmazása
    filtered_image = cv2.medianBlur(image, kernel_size)
    
    return filtered_image

# Kernel méretek léptetése
start = 3
stop = 101
step = 2
kernel_sizes = list(range(start, stop+1, step))

# Kimeneti mappa létrehozása
output_dir = 'median_filter'
os.makedirs(output_dir, exist_ok=True)

# Mediánszűrő alkalmazása különböző kernel méretekkel
filtered_images = []
for kernel_size in kernel_sizes:
    filtered_image = median_filter(img, kernel_size)
    filtered_images.append(filtered_image)
    
    # Eredmények mentése fájlba
    file_name = f'{kernel_size}x{kernel_size}.png'
    file_path = os.path.join(output_dir, file_name)
    cv2.imwrite(file_path, filtered_image)



# Lokális küszöbölés

In [ ]:

def local_threshold_mean(image, kernel_size):
    # Átlagszűrő alkalmazása a küszöbérték meghatározásához
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    
    # Konvolúció az eredeti képpel
    convolved_image = cv2.filter2D(image, -1, np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size * kernel_size))
    
    # Lokális küszöbölés az átlag alapján
    thresholded_image = np.where(image > convolved_image, 255, 0).astype(np.uint8)
    
    return thresholded_image

# Lokális küszöbölés alkalmazása
kernel_size = 25
thresholded_image = local_threshold_mean(grey_img, kernel_size)

# Eredmények megjelenítése
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(grey_img, cmap='gray')
axs[0].set_title('Eredeti kép')
axs[0].axis('off')

axs[1].imshow(thresholded_image, cmap='gray')
axs[1].set_title('Lokális küszöbölés átlag alapján')
axs[1].axis('off')

plt.show()


# Élkiemelés
## Roberts filter

In [ ]:
lines = cv2.imread('Images/lines.png', cv2.IMREAD_GRAYSCALE)

roberts_x = np.array([[1, 0], [0, -1]])
roberts_y = np.array([[0, 1], [-1, 0]])

filtered_image_x = cv2.filter2D(lines, -1, roberts_x)
filtered_image_y = cv2.filter2D(lines, -1, roberts_y)

# Gradiens vektorok összege
gradient_magnitude = np.sqrt(filtered_image_x**2 + filtered_image_y**2)

# Eredmények megjelenítése
plt.subplot(2, 2, 1)
plt.imshow(lines, cmap='gray')
plt.axis('off')
plt.title('Eredeti kép')

plt.subplot(2, 2, 2)
plt.imshow(filtered_image_x, cmap='gray')
plt.axis('off')
plt.title('Roberts (X irány)')

plt.subplot(2, 2, 3)
plt.imshow(filtered_image_y, cmap='gray')
plt.axis('off')
plt.title('Roberts (Y irány)')

plt.subplot(2, 2, 4)
plt.imshow(gradient_magnitude, cmap='gray')
plt.axis('off')
plt.title('Gradiens vektor hossza')

plt.tight_layout()
plt.show()


## Sobel filter

In [ ]:
# Sobel operátor alkalmazása az X és Y irányban
filtered_image_sobel_x = cv2.Sobel(lines, cv2.CV_64F, 1, 0, ksize=3)
filtered_image_sobel_y = cv2.Sobel(lines, cv2.CV_64F, 0, 1, ksize=3)

# Gradiens magnitúdó számítása
gradient_magnitude = np.sqrt(filtered_image_sobel_x**2 + filtered_image_sobel_y**2)

# Eredmények megjelenítése
plt.subplot(3, 1, 1)
plt.imshow(filtered_image_sobel_x, cmap='gray')
plt.axis('off')
plt.title('Sobel (X irány)')

plt.subplot(3, 1, 2)
plt.imshow(filtered_image_sobel_y, cmap='gray')
plt.axis('off')
plt.title('Sobel (Y irány)')

plt.subplot(3, 1, 3)
plt.imshow(gradient_magnitude, cmap='gray')
plt.axis('off')
plt.title('Sobel (Gradiens magnitúdó)')

plt.tight_layout()
plt.show()

# Élkiemelés konvolúciós mátrixszokkal

In [ ]:
def edge_detection(image, kernel):
    # Élkimelés konvolúcióval
    filtered_image = cv2.filter2D(image, -1, kernel)
    
    return filtered_image

def compute_gradient_magnitude(image):
    # Gradiensvektorok számítása
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # Gradiens vektorok hosszának kiszámítása
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

    return gradient_x, gradient_y, gradient_magnitude

lines = cv2.imread('Images/lines.png', cv2.IMREAD_GRAYSCALE)

roberts_x = np.array([[1, 0],
                      [0, -1]])

roberts_y = np.array([[0, 1],
                      [-1, 0]])

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

sobel_y = np.array([[-1, -2, -1],
                    [0, 0, 0],
                    [1, 2, 1]])

# Élkimelés Roberts operátorokkal
roberts_x_filtered = edge_detection(lines, roberts_x)
roberts_y_filtered = edge_detection(lines, roberts_y)

# Élkimelés Sobel operátorokkal
sobel_x_filtered = edge_detection(lines, sobel_x)
sobel_y_filtered = edge_detection(lines, sobel_y)

# Gradiens vektorok hosszának számítása
roberts_x_gradient, roberts_y_gradient, roberts_magnitude = compute_gradient_magnitude(lines)
sobel_x_gradient, sobel_y_gradient, sobel_magnitude = compute_gradient_magnitude(lines)

# Eredmények megjelenítése
fig, axs = plt.subplots(4, 3, figsize=(12, 12))

axs[0, 0].imshow(roberts_x_filtered, cmap='gray')
axs[0, 0].set_title('Roberts x')
axs[0, 0].axis('off')

axs[0, 1].imshow(roberts_y_filtered, cmap='gray')
axs[0, 1].set_title('Roberts y')
axs[0, 1].axis('off')

axs[0, 2].imshow(roberts_magnitude, cmap='gray')
axs[0, 2].set_title('Roberts gradiens magnitúdó')
axs[0, 2].axis('off')

axs[1, 0].imshow(sobel_x_filtered, cmap='gray')
axs[1, 0].set_title('Sobel x')
axs[1, 0].axis('off')

axs[1, 1].imshow(sobel_y_filtered, cmap='gray')
axs[1, 1].set_title('Sobel y')
axs[1, 1].axis('off')

axs[1, 2].imshow(sobel_magnitude, cmap='gray')
axs[1, 2].set_title('Sobel gradiens magnitúdó')
axs[1, 2].axis('off')

axs[2, 0].imshow(roberts_x_gradient, cmap='gray')
axs[2, 0].set_title('Roberts x gradiens')
axs[2, 0].axis('off')

axs[2, 1].imshow(roberts_y_gradient, cmap='gray')
axs[2, 1].set_title('Roberts y gradiens')
axs[2, 1].axis('off')

axs[2, 2].imshow(sobel_x_gradient, cmap='gray')
axs[2, 2].set_title('Sobel x gradiens')
axs[2, 2].axis('off')

axs[3, 0].imshow(sobel_y_gradient, cmap='gray')
axs[3, 0].set_title('Sobel y gradiens')
axs[3, 0].axis('off')

axs[3, 1].axis('off')
axs[3, 2].axis('off')

plt.tight_layout()
plt.show()


# Blobdetektálás

In [ ]:
# Binarizálás
threshold_value = 160
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

# Kontúrkeresés a binarizált képen
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Szűrés a kontúrokon a kívánt kritériumok alapján (pl. terület)
min_area = 100  # Minimum területi küszöb
filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_area]

# Blobok megjelölése a képen
image_with_blobs = cv2.drawContours(img.copy(), filtered_contours, -1, (0, 255, 0), 2)

# Blobok tulajdonságainak gyűjtése (pl. középpont, terület)
blobs = []
for contour in filtered_contours:
    moments = cv2.moments(contour)
    area = cv2.contourArea(contour)
    centroid_x = int(moments["m10"] / moments["m00"])
    centroid_y = int(moments["m01"] / moments["m00"])
    blobs.append({"area": area, "centroid": (centroid_x, centroid_y)})

# Eredmények megjelenítése
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cv2.cvtColor(image_with_blobs, cv2.COLOR_BGR2RGB))
ax.set_title('Blobdetektálás')
ax.axis('off')

plt.show()


# Vonalkövetés

In [ ]:
def follow_line(image, start_point, num_steps):
    current_point = start_point
    line_points = [current_point]

    for _ in range(num_steps):
        x, y = current_point
        neighborhood = [(x-1, y-1), (x, y-1), (x+1, y-1), (x-1, y), (x+1, y), (x-1, y+1), (x, y+1), (x+1, y+1)]

        next_point = None
        min_value = float('inf')

        for neighbor in neighborhood:
            nx, ny = neighbor
            if nx >= 0 and ny >= 0 and nx < image.width and ny < image.height:
                value = image.getpixel((nx, ny))
                if value < min_value:
                    min_value = value
                    next_point = neighbor

        if next_point is None:
            break

        current_point = next_point
        line_points.append(current_point)

    return line_points

# Kép beolvasása
image = Image.open("Images/lines.png").convert("L")  # Átalakítás szürkeárnyalatos képre

# Kezdőpont meghatározása
start_point = (500, 500)

# Lépésszám meghatározása
num_steps = 100

# Vonalkövetés
line_points = follow_line(image, start_point, num_steps)

# Eredmény megjelenítése
plt.imshow(image, cmap='gray')
plt.plot(*zip(*line_points), color='red', linewidth=2)
plt.show()


'''Ez a program egy vonalkövető algoritmust valósít meg, 
amely egy adott képen egy kiindulópontból egy adott számú lépést tesz meg, és végigköveti a vonalat. 
A vonalkövetés során az algoritmus minden lépésben kiválasztja a legközelebbi szomszédos pontot, 
amelynek pixelértéke a legalacsonyabb. 
Az algoritmus ezáltal megtalálja a vonal irányát és az általa megtett pontokat piros vonallal ábrázolja
az eredeti képen.'''